In [93]:
import numpy as np
import pandas as pd
import sys

In [94]:
sys.path.append('/usr/local/Cellar/graph-tool/2.53/lib/python3.11/site-packages')

In [95]:
import graph_tool 
from graph_tool.all import *

In [96]:
s = pd.read_csv("whole_cell_s_matrix (1).csv")

In [97]:
s

,Unnamed: 0,1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse),1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse),1PFRUCTPHOSN-RXN,1TRANSKETO-RXN (reverse),2-DEHYDROPANTOATE-REDUCT-RXN (reverse),2-ISOPROPYLMALATESYN-RXN,2-OCTAPRENYL-6-METHOXYPHENOL-HYDROX-RXN,2-OCTAPRENYL-6-OHPHENOL-METHY-RXN,2-OCTAPRENYL-METHOXY-BENZOQ-METH-RXN,...,ISOCHORSYN-RXN__MENF-CPLX,RXN-9535__FABB-CPLX,RXN0-313__CPLX0-201 (reverse),RXN0-384__H2NEOPTERINP3PYROPHOSPHOHYDRO-MONOMER,RXN0-5186__G7408-MONOMER,SERINE-O-ACETTRAN-RXN__CPLX0-237,SERINE-O-ACETTRAN-RXN__CYSSYNMULTI-CPLX,UNDECAPRENYL-DIPHOSPHATASE-RXN[CCO-CYTOSOL]-UNDECAPRENYL-DIPHOSPHATE/WATER//CPD-9646/Pi/PROTON.64.__PGPPHOSPHAB-MONOMER,URPHOS-RXN__URPHOS-CPLX,maintenance_reaction
0,NAD[c],1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NADH[c],-1,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PROTON[c],-1,-1,1,0,-1,1,-1,1,1,...,0,-1,0,1,0,0,0,1,0,1
3,GLUCONATE[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NADP[c],0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,R-3-hydroxymyristoyl-ACPs[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
485,Adenylated-ThiS-Proteins[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
486,ThiS-CoASH-proteins[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
487,Thi-S[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
flow = pd.read_csv("whole_cell_flow_data.csv")

In [115]:
# Make edges
edges = []
for column in s:
    if column == "Unnamed: 0":
        continue
    for i, item in s[column].items():
        if item < 0:
            edge = (s["Unnamed: 0"][i], column)
            edges.append(edge)
        elif item > 0:
            edge = (column, s["Unnamed: 0"][i])
            edges.append(edge)

In [116]:
g = Graph(edges, hashed=True)
degrees = g.degree_property_map("out")

In [117]:
max_degree_vertices = sorted(range(g.num_vertices()), key=lambda i: degrees[i], reverse=True)[:50]
for v in max_degree_vertices:
    g.remove_vertex(v)

In [118]:
pos = graph_tool.draw.sfdp_layout(g, C=100)
graph_draw(g, pos=pos, output="graph-draw-a.pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x174db4bd0, at 0x16d339a90>

In [101]:
print(edges)

[('1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse)', 'NAD[c]'), ('NADH[c]', '1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse)'), ('PROTON[c]', '1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse)'), ('1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse)', '5-METHYL-THF[c]'), ('METHYLENE-THF[c]', '1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse)'), ('1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse)', 'NAD[c]'), ('NADH[c]', '1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse)'), ('PROTON[c]', '1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse)'), ('1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse)', 'CPD-1302[c]'), ('CPD-12996[c]', '1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse)'), ('1PFRUCTPHOSN-RXN', 'PROTON[c]'), ('ATP[c]', '1PFRUCTPHOSN-RXN'), ('FRU1P[c]', '1PFRUCTPHOSN-RXN'), ('1PFRUCTPHOSN-RXN', 'ADP[c]'), ('1PFRUC

In [21]:
flow

,Unnamed: 0,data
0,1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/N...,3851.0
1,1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROT...,16095.0
2,1PFRUCTPHOSN-RXN,1238118.0
3,1TRANSKETO-RXN (reverse),7974.0
4,2-DEHYDROPANTOATE-REDUCT-RXN (reverse),1131.0
...,...,...
465,SERINE-O-ACETTRAN-RXN__CPLX0-237,17162.0
466,SERINE-O-ACETTRAN-RXN__CYSSYNMULTI-CPLX,16186.0
467,UNDECAPRENYL-DIPHOSPHATASE-RXN[CCO-CYTOSOL]-UN...,143.0
468,URPHOS-RXN__URPHOS-CPLX,3.0


In [22]:
g = Graph()